In [5]:
import math
from math import radians, cos, sin, asin, acos, sqrt, pi
import pandas as pd
import numpy as np
from datetime import time
import datetime as dt
from datetime import datetime as datetime
from datetime import datetime as datetime
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.interpolate import interp1d
import seaborn as sns

In [26]:
# Loading Cleaned data
analysis_test = pd.read_excel(r"C:\Users\deepak\Desktop\Datathon\Analysis_data.xlsx",sheet_name='Analysis_test')
analysis_test

,ID,time,RR,VO2,power,HR,HR_mean,VO2_mean
0,1,-130.840,672.0,0.302008,0,89.285714,89.285714,0.302008
1,1,-130.168,672.0,0.302008,0,89.285714,89.285714,0.302008
2,1,-129.536,632.0,0.301336,0,94.936709,91.169379,0.301784
3,1,-109.280,672.0,0.310178,0,89.285714,90.698463,0.303882
4,1,-104.280,660.0,0.329340,0,90.909091,90.740589,0.308974
...,...,...,...,...,...,...,...,...
47896,18,947.288,536.0,0.507071,0,111.940299,113.483928,0.481351
47897,18,947.816,528.0,0.507071,0,113.636364,112.955387,0.487781
47898,18,948.344,528.0,0.507071,0,113.636364,113.126269,0.494211
47899,18,948.884,540.0,0.507071,0,111.111111,112.792100,0.500641


In [27]:

# Calculate power_idx using rleid
analysis_test['power_idx'] = analysis_test.groupby('ID')['power'].apply(lambda x: (x != x.shift()).cumsum())

# Create a temporary DataFrame to find the last power_idx for each ID
tmp = analysis_test.groupby('ID')['power_idx'].max().reset_index()
tmp.rename(columns={'power_idx': 'last_power_idx'}, inplace=True)

# Merge tmp DataFrame with analysis_test to update the 'test' column
analysis_test = pd.merge(analysis_test, tmp, on='ID', how='left')

# Use case logic to update the 'test' column based on conditions
analysis_test['test'] = analysis_test.apply(
    lambda row: 'rest' if row['power_idx'] == 1
    else ('recovery' if row['power_idx'] == row['last_power_idx'] else 'test'),
    axis=1
)

# Drop the 'last_power_idx' column if it's no longer needed
analysis_test.drop('last_power_idx', axis=1, inplace=True)
analysis_test


,ID,time,RR,VO2,power,HR,HR_mean,VO2_mean,power_idx,test
0,1,-130.840,672.0,0.302008,0,89.285714,89.285714,0.302008,1,rest
1,1,-130.168,672.0,0.302008,0,89.285714,89.285714,0.302008,1,rest
2,1,-129.536,632.0,0.301336,0,94.936709,91.169379,0.301784,1,rest
3,1,-109.280,672.0,0.310178,0,89.285714,90.698463,0.303882,1,rest
4,1,-104.280,660.0,0.329340,0,90.909091,90.740589,0.308974,1,rest
...,...,...,...,...,...,...,...,...,...,...
47896,18,947.288,536.0,0.507071,0,111.940299,113.483928,0.481351,13,recovery
47897,18,947.816,528.0,0.507071,0,113.636364,112.955387,0.487781,13,recovery
47898,18,948.344,528.0,0.507071,0,113.636364,113.126269,0.494211,13,recovery
47899,18,948.884,540.0,0.507071,0,111.111111,112.792100,0.500641,13,recovery


In [28]:
import pandas as pd

# Sort the DataFrame by 'ID' and 'time'
analysis_test.sort_values(by=['ID', 'time'], inplace=True)

# Create a mask for rows where 'test' is "recovery"
recovery_mask = analysis_test['test'] == "recovery"

# Calculate the first "recovery" timestamp for each 'ID'
first_recovery_timestamp = analysis_test[recovery_mask].groupby('ID')['time'].first()

# Reset the index to match 'ID'
first_recovery_timestamp = first_recovery_timestamp.reset_index()

# Merge the first recovery timestamp back into the main DataFrame
analysis_test = pd.merge(analysis_test, first_recovery_timestamp, on='ID', suffixes=('', '_first_recovery'))

# Calculate 'time_rest' by subtracting the first recovery timestamp from 'time'
analysis_test['time_rest'] = analysis_test['time'] - analysis_test['time_first_recovery']

# Drop the temporary column 'time_first_recovery'
analysis_test.drop('time_first_recovery', axis=1, inplace=True)
analysis_test

,ID,time,RR,VO2,power,HR,HR_mean,VO2_mean,power_idx,test,time_rest
0,1,-130.840,672.0,0.302008,0,89.285714,89.285714,0.302008,1,rest,-785.020
1,1,-130.168,672.0,0.302008,0,89.285714,89.285714,0.302008,1,rest,-784.348
2,1,-129.536,632.0,0.301336,0,94.936709,91.169379,0.301784,1,rest,-783.716
3,1,-109.280,672.0,0.310178,0,89.285714,90.698463,0.303882,1,rest,-763.460
4,1,-104.280,660.0,0.329340,0,90.909091,90.740589,0.308974,1,rest,-758.460
...,...,...,...,...,...,...,...,...,...,...,...
47896,18,947.288,536.0,0.507071,0,111.940299,113.483928,0.481351,13,recovery,199.284
47897,18,947.816,528.0,0.507071,0,113.636364,112.955387,0.487781,13,recovery,199.812
47898,18,948.344,528.0,0.507071,0,113.636364,113.126269,0.494211,13,recovery,200.340
47899,18,948.884,540.0,0.507071,0,111.111111,112.792100,0.500641,13,recovery,200.880


In [29]:
import pandas as pd

# Filter the rows where 'test' is "recovery"
recovery_data = analysis_test[analysis_test['test'] == "recovery"]

# Calculate HRR120 and HRR60 based on the conditions and group by 'ID'
HRR = recovery_data.groupby('ID').apply(lambda group:
    pd.Series({
        'HRR120': group['HR_mean'].iloc[0] - group[group['time_rest'] > 120]['HR_mean'].iloc[0],
        'HRR60': group['HR_mean'].iloc[0] - group[group['time_rest'] > 60]['HR_mean'].iloc[0]
    })
).reset_index()

# Rename the index column to 'ID' for the final result
HRR.rename(columns={'level_1': 'ID'}, inplace=True)

analysis_test = pd.merge(analysis_test, HRR, on='ID', how='left')
analysis_test

,ID,time,RR,VO2,power,HR,HR_mean,VO2_mean,power_idx,test,time_rest,HRR120,HRR60
0,1,-130.840,672.0,0.302008,0,89.285714,89.285714,0.302008,1,rest,-785.020,48.615856,21.917749
1,1,-130.168,672.0,0.302008,0,89.285714,89.285714,0.302008,1,rest,-784.348,48.615856,21.917749
2,1,-129.536,632.0,0.301336,0,94.936709,91.169379,0.301784,1,rest,-783.716,48.615856,21.917749
3,1,-109.280,672.0,0.310178,0,89.285714,90.698463,0.303882,1,rest,-763.460,48.615856,21.917749
4,1,-104.280,660.0,0.329340,0,90.909091,90.740589,0.308974,1,rest,-758.460,48.615856,21.917749
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47896,18,947.288,536.0,0.507071,0,111.940299,113.483928,0.481351,13,recovery,199.284,62.815554,41.384889
47897,18,947.816,528.0,0.507071,0,113.636364,112.955387,0.487781,13,recovery,199.812,62.815554,41.384889
47898,18,948.344,528.0,0.507071,0,113.636364,113.126269,0.494211,13,recovery,200.340,62.815554,41.384889
47899,18,948.884,540.0,0.507071,0,111.111111,112.792100,0.500641,13,recovery,200.880,62.815554,41.384889


In [30]:
analysis_test.drop(['time','RR','VO2','power','HR','HR_mean','VO2_mean'], axis=1, inplace=True)
analysis_test

,ID,power_idx,test,time_rest,HRR120,HRR60
0,1,1,rest,-785.020,48.615856,21.917749
1,1,1,rest,-784.348,48.615856,21.917749
2,1,1,rest,-783.716,48.615856,21.917749
3,1,1,rest,-763.460,48.615856,21.917749
4,1,1,rest,-758.460,48.615856,21.917749
...,...,...,...,...,...,...
47896,18,13,recovery,199.284,62.815554,41.384889
47897,18,13,recovery,199.812,62.815554,41.384889
47898,18,13,recovery,200.340,62.815554,41.384889
47899,18,13,recovery,200.880,62.815554,41.384889


In [31]:
analysis_test.to_excel("C:/Users/deepak/Desktop/Datathon/HRV_Analysis_test.xlsx")